Users

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [ ]:
num_users=50000
user_ids=[f"U{i:06d}"for i in range(1,num_users+1)]

In [ ]:
devices=np.random.choice(['ios','andriod','web'],size=num_users,p=[0.6,0.3,0.1])

In [ ]:
pd.Series(devices).value_counts(normalize=True)

In [ ]:
ages=np.random.randint(18,61,size=num_users)

In [ ]:
start_date = datetime(2024, 1, 1)

# random days between 0 and 365
signup_offsets = np.random.randint(0, 366, size=num_users)

signup_dates = [start_date + timedelta(days=int(d)) for d in signup_offsets]

In [ ]:
phone_numbers = ['07' + ''.join(np.random.choice(list('0123456789'), size=9)) for _ in range(num_users)]

In [ ]:
pincodes = np.random.choice(
    ['SW1', 'SW3', 'E14', 'W1', 'SE1', 'EC2', 'N1', 'IG1', 'CR0', 'KT1'],
    size=num_users
)

In [ ]:
first_names = ['Aisha', 'Varsha', 'Riya', 'Sandeep', 'James', 'Amir', 'Leena', 'Priya', 'Tom', 'Hannah']
last_names  = ['Patel', 'Khan', 'Smith', 'Williams', 'Brown', 'Shah', 'Singh', 'Jones', 'Taylor', 'Ahmed']

names = [
    f"{np.random.choice(first_names)} {np.random.choice(last_names)}"
    for _ in range(num_users)
]

In [ ]:
users = pd.DataFrame({
    'user_id': user_ids,
    'name': names,
    'phone': phone_numbers,
    'pincode': pincodes,
    'signup_date': signup_dates,
    'age': ages,
    'device': devices
})

In [ ]:
print(users.head())
print(users.info())

In [ ]:
users.shape

In [ ]:
users.device.value_counts().head()

In [ ]:
users.sample(1)

TRANSACTIONS

In [ ]:
users.to_csv("users.csv", index=False)

In [ ]:
import os
os.listdir()

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

BASE_DIR = '/content/drive/MyDrive/monzo_user_analytics'  # change if you like
import os
os.makedirs(f'{BASE_DIR}/data', exist_ok=True)

# Example saves:
users.to_csv(f'{BASE_DIR}/data/users.csv', index=False)
#transactions.to_csv(f'{BASE_DIR}/data/transactions.csv', index=False)

In [ ]:
import pandas as pd
check = pd.read_csv(f'{BASE_DIR}/data/users.csv', parse_dates=['signup_date'])
check.head()

In [ ]:
import numpy as np
from datetime import datetime
import os

# Paths (we’ll keep the same BASE_DIR you already set earlier)
TXNS_PATH = f"{BASE_DIR}/data/transactions.csv"
os.makedirs(f"{BASE_DIR}/data", exist_ok=True)

# --- STEP 1: Read users again to access their signup dates ---
users = pd.read_csv(f"{BASE_DIR}/data/users.csv", parse_dates=["signup_date"])
user_ids = users["user_id"].to_numpy()
signups = users["signup_date"].to_numpy("datetime64[D]")
n_users = len(user_ids)

rng = np.random.default_rng(42)  # for reproducibility

# --- STEP 2: Decide how many transactions per user (light, medium, heavy) ---
u = rng.random(n_users)
txn_counts = np.empty(n_users, dtype=np.int32)
txn_counts[u < 0.6] = rng.integers(1, 21, size=(u < 0.6).sum())      # light
txn_counts[(u >= 0.6) & (u < 0.9)] = rng.integers(20, 81, size=((u >= 0.6) & (u < 0.9)).sum())  # medium
txn_counts[u >= 0.9] = rng.integers(80, 201, size=(u >= 0.9).sum())  # heavy

total_txns = int(txn_counts.sum())
print(f"Total transactions to generate: {total_txns:,}")

# --- STEP 3: Repeat user info per transaction (vectorized, no loops) ---
rep_user_ids = np.repeat(user_ids, txn_counts)
rep_signups  = np.repeat(signups,  txn_counts)

# --- STEP 4: Generate transaction fields ---
day_offsets = rng.integers(0, 366, size=total_txns)        # days after signup
txn_dates   = rep_signups + day_offsets.astype("timedelta64[D]")
txn_dates   = np.minimum(txn_dates, np.datetime64("2024-12-31"))

amounts = np.round(rng.exponential(scale=35, size=total_txns) + 1.0, 2)  # skewed realistic spends
categories = np.array(["groceries", "transport", "bills", "shopping", "restaurants", "entertainment"])
channels   = np.array(["card", "online", "direct_debit"])
txn_cats   = categories[rng.integers(0, len(categories), size=total_txns)]
txn_chans  = channels[rng.integers(0, len(channels),   size=total_txns)]

# --- STEP 5: Build DataFrame ---
transactions = pd.DataFrame({
    "user_id": rep_user_ids,
    "txn_date": pd.to_datetime(txn_dates),
    "amount": amounts,
    "category": txn_cats,
    "channel": txn_chans,
})

transactions.sort_values(["user_id", "txn_date"], inplace=True, kind="mergesort")
transactions.reset_index(drop=True, inplace=True)
transactions.insert(0, "txn_id", np.arange(1, len(transactions) + 1, dtype=np.int64))

# --- STEP 6: Save & verify ---
transactions.to_csv(TXNS_PATH, index=False)
print(f"\nSaved transactions to: {TXNS_PATH}")

# --- STEP 7: Quick checks ---
print("\n--- Sanity checks ---")
print("shape:", transactions.shape)
print("date range:", transactions["txn_date"].min(), "→", transactions["txn_date"].max())
print("unique users:", transactions["user_id"].nunique())
print("\namounts:\n", transactions["amount"].describe())
print("\ntransactions per user:\n", transactions.groupby("user_id").size().describe())

In [ ]:

# Paths
FEATURES_PATH = f"{BASE_DIR}/data/feature_usage.csv"
USERS_PATH    = f"{BASE_DIR}/data/users.csv"

os.makedirs(f"{BASE_DIR}/data", exist_ok=True)

# Load users (need user_id, signup_date, device, age)
users = pd.read_csv(USERS_PATH, parse_dates=["signup_date"])
user_ids = users["user_id"].to_numpy()
signups  = users["signup_date"].to_numpy("datetime64[D]")
devices  = users["device"].to_numpy()
ages     = users["age"].to_numpy()
n_users  = len(users)

rng = np.random.default_rng(123)

# Define features
features = np.array(["pots", "get_paid_early", "budgeting", "roundups", "investments", "pensions"])

# --- 1) Per-user adoption probabilities (simple, interpretable rules) ---
# Start from a base probability per feature
base_p = {
    "pots":            0.45,
    "get_paid_early":  0.35,
    "budgeting":       0.40,
    "roundups":        0.30,
    "investments":     0.15,
    "pensions":        0.10,
}

# Device adjustments (e.g., iOS users slightly more likely to adopt budgeting; Web less)
device_adj = np.zeros(n_users)
device_adj[devices == "iOS"]     =  +0.05
device_adj[devices == "Android"] =  +0.00
device_adj[devices == "Web"]     =  -0.05

# Age adjustments (older slightly more likely to adopt pensions/investments)
age_adj_invest   = np.clip((ages - 25) / 50, 0, 0.10)   # up to +0.10
age_adj_pensions = np.clip((ages - 30) / 40, 0, 0.12)   # up to +0.12

# Build a (n_users x n_features) matrix of adoption probabilities
P = np.zeros((n_users, len(features)), dtype=float)
for j, f in enumerate(features):
    p = base_p[f] + device_adj
    if f == "investments":
        p = p + age_adj_invest
    if f == "pensions":
        p = p + age_adj_pensions
    P[:, j] = np.clip(p, 0.01, 0.95)  # ensure valid prob bounds

# --- 2) Sample which features each user adopts (Bernoulli trials) ---
adopt_matrix = rng.random(P.shape) < P   # True if adopted
adopt_counts = adopt_matrix.sum(axis=1)  # features adopted per user (for sanity)

# --- 3) For each adopted feature, generate a number of usage events ---
# Usage intensity by feature (mean events per adopted feature)
feature_mean_uses = {
    "pots":            8,
    "get_paid_early":  6,
    "budgeting":       10,
    "roundups":        20,
    "investments":     4,
    "pensions":        2,
}

# Build counts of usage events per (user, feature)
usage_rows_user   = []
usage_rows_feat   = []
usage_rows_date   = []
usage_rows_action = []  # "enable" or "use"

# Precompute mean array aligned with features
means = np.array([feature_mean_uses[f] for f in features], dtype=float)

# We vectorize by:
# 1) get indices where feature is adopted
u_idx, f_idx = np.where(adopt_matrix)  # arrays of same length
n_adopt_events = u_idx.size

# 2) Sample #uses per adopted feature via Poisson (>=1), clipped
uses = rng.poisson(lam=means[f_idx]) + 1
uses = np.clip(uses, 1, 60)

# 3) For each (user, feature), create 1 "enable" event near signup + N "use" events spread across 2024
# We'll do this in chunks to avoid huge Python loops; but we still need to expand variable lengths.
# Chunk size controls memory usage; adjust if needed.
chunk = 200_000
for start in range(0, n_adopt_events, chunk):
    end = min(start + chunk, n_adopt_events)

    u_block = u_idx[start:end]
    f_block = f_idx[start:end]
    k_block = uses[start:end]

    # ENABLE events (one per adopted feature)
    enable_dates = signups[u_block] + rng.integers(0, 30, size=len(u_block)).astype("timedelta64[D]")
    enable_dates = np.minimum(enable_dates, np.datetime64("2024-12-31"))

    usage_rows_user.extend(user_ids[u_block])
    usage_rows_feat.extend(features[f_block])
    usage_rows_date.extend(enable_dates.astype("datetime64[ns]"))
    usage_rows_action.extend(["enable"] * len(u_block))

    # USE events (variable counts) -> expand each row by k_block
    rep_users = np.repeat(user_ids[u_block], k_block)
    rep_feats = np.repeat(features[f_block], k_block)
    rep_signup = np.repeat(signups[u_block], k_block)

    day_offsets = rng.integers(0, 366, size=rep_users.size)
    use_dates = rep_signup + day_offsets.astype("timedelta64[D]")
    use_dates = np.minimum(use_dates, np.datetime64("2024-12-31"))

    usage_rows_user.extend(rep_users)
    usage_rows_feat.extend(rep_feats)
    usage_rows_date.extend(use_dates.astype("datetime64[ns]"))
    usage_rows_action.extend(["use"] * rep_users.size)

# --- 4) Assemble DataFrame ---
feature_usage = pd.DataFrame({
    "user_id": usage_rows_user,
    "feature": usage_rows_feat,
    "event_date": pd.to_datetime(usage_rows_date),
    "action": usage_rows_action,
})

feature_usage.sort_values(["user_id", "event_date", "feature", "action"], inplace=True, kind="mergesort")
feature_usage.reset_index(drop=True, inplace=True)
feature_usage.insert(0, "event_id", np.arange(1, len(feature_usage) + 1, dtype=np.int64))

# --- 5) Save & sanity checks ---
feature_usage.to_csv(FEATURES_PATH, index=False)
print(f"Saved: {FEATURES_PATH}")

print("\n--- Sanity checks ---")
print("shape:", feature_usage.shape)
print("date range:", feature_usage["event_date"].min(), "→", feature_usage["event_date"].max())
print("unique users in features:", feature_usage["user_id"].nunique())
print("\nCounts by action:\n", feature_usage["action"].value_counts())
print("\nTop features by events:\n", feature_usage["feature"].value_counts().head(10))
print("\nAdopted features per user (mean):", adopt_counts.mean().round(2))

 Monthly Active Users (MAU)

In [ ]:
tx = pd.read_csv(f"{BASE_DIR}/data/transactions.csv", parse_dates=["txn_date"])
mau = (tx
       .assign(month=tx["txn_date"].dt.to_period("M"))
       .groupby("month")["user_id"].nunique()
       .rename("MAU")
       .to_frame()
       .reset_index())
mau["month"] = mau["month"].astype(str)
mau.head(), mau.tail()

Feature adoption rate (what % of users enabled each feature)

In [ ]:
fu = pd.read_csv(f"{BASE_DIR}/data/feature_usage.csv", parse_dates=["event_date"])
adoption = (fu[fu["action"]=="enable"]
            .groupby("feature")["user_id"].nunique()
            .rename("adopters")
            .to_frame()
            .reset_index())
adoption["adoption_rate"] = adoption["adopters"] / users.shape[0]
adoption.sort_values("adoption_rate", ascending=False)

Spend by device (are iOS users spending more?)

In [ ]:
tx_users = tx.merge(users[["user_id","device"]], on="user_id", how="left")
spend_by_device = (tx_users
                   .groupby("device")["amount"]
                   .agg(["count","mean","sum"])
                   .rename(columns={"count":"txn_cnt","mean":"avg_txn","sum":"total_spend"})
                   .reset_index())
spend_by_device

New vs old users

In [ ]:
users["signup_month"] = users["signup_date"].dt.to_period("M").astype(str)
tx_users = tx.merge(users[["user_id","signup_month"]], on="user_id", how="left")
spend_by_signup_month = (tx_users
                         .assign(tx_month=tx_users["txn_date"].dt.to_period("M").astype(str))
                         .groupby(["signup_month","tx_month"])["user_id"]
                         .nunique()
                         .rename("active_users")
                         .reset_index())
spend_by_signup_month.head()

In [ ]:
# Convert your earlier MAU calculation to a proper plot
plt.figure(figsize=(8,5), dpi=120)
plt.plot(mau["month"], mau["MAU"], marker='o', color='purple')
plt.title("Monthly Active Users (MAU) Over Time")
plt.ylabel("Active Users")
plt.xlabel("Month (2024)")
plt.xticks(rotation=45)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
cohort = spend_by_signup_month[spend_by_signup_month["signup_month"] == "2024-01"]

plt.figure(figsize=(8,5), dpi=120)
plt.plot(cohort["tx_month"], cohort["active_users"], marker='o', color='teal')
plt.title("Cohort Retention: Users Signed Up in Jan 2024")
plt.ylabel("Active Users (same cohort)")
plt.xlabel("Month of Activity")
plt.xticks(rotation=45)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = '/content/drive/MyDrive/banking-user-analysis'
import os; os.makedirs(f'{BASE_DIR}/data', exist_ok=True)
os.makedirs(f'{BASE_DIR}/images', exist_ok=True)
os.makedirs(f'{BASE_DIR}/notebooks', exist_ok=True)

In [ ]:
users.to_csv(f'{BASE_DIR}/data/users.csv', index=False)
transactions.to_csv(f'{BASE_DIR}/data/transactions.csv', index=False)
feature_usage.to_csv(f'{BASE_DIR}/data/feature_usage.csv', index=False)

In [ ]:
plt.savefig(f'{BASE_DIR}/images/feature_adoption.png', bbox_inches='tight')
# do similar for other charts:
# spend_by_device.png, mau_trend.png, cohort_retention.png

In [ ]:
readme = """<paste the README content here>"""
with open(f'{BASE_DIR}/README.md','w') as f:
    f.write(readme)

In [ ]:
import pandas as pd
tx_small = transactions.sample(n=250000, random_state=42)  # tweak n to keep <100MB
tx_small.to_csv(f'{BASE_DIR}/data/transactions_sample.csv', index=False)

In [ ]:
transactions['month'] = transactions['txn_date'].dt.to_period('M').astype(str)
for m, dfm in transactions.groupby('month'):
    dfm.drop(columns='month').to_csv(f'{BASE_DIR}/data/transactions_{m}.csv', index=False)

to accomodate github commit

In [ ]:
import pandas as pd

# load your originals from Drive
users = pd.read_csv(f'{BASE_DIR}/data/users.csv')
transactions = pd.read_csv(f'{BASE_DIR}/data/transactions.csv')
feature_usage = pd.read_csv(f'{BASE_DIR}/data/feature_usage.csv')

# create smaller versions (adjust n as needed)
users_sample = users.sample(n=2000, random_state=42)
transactions_sample = transactions.sample(n=50000, random_state=42)
feature_usage_sample = feature_usage.sample(n=100000, random_state=42)

# save them with _sample suffix
users_sample.to_csv(f'{BASE_DIR}/data/users_sample.csv', index=False)
transactions_sample.to_csv(f'{BASE_DIR}/data/transactions_sample.csv', index=False)
feature_usage_sample.to_csv(f'{BASE_DIR}/data/feature_usage_sample.csv', index=False)